<a href="https://colab.research.google.com/github/mahamapara/Predicting-food-loss-for-climate-change/blob/main/Final_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Modules/packages required</h3>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as pyplot
from sklearn import linear_model
import statsmodels.api as sm
from pandas.api.types import is_string_dtype

<h3>Loading, exploring, manipulating the data</h3>

In [ ]:
#reads the data
fw = pd.read_csv("foodwaste.csv") 

In [ ]:
#gives datatypes of each column in the dataframe
fw.dtypes

In [ ]:
#looking at the data
fw.head()

In [ ]:
#total number of missing values in the dataset and dim of dataset
fw.isnull().sum().sum()
fw.shape

In [ ]:
#total number of missing values in the causeofloss feature
fw['causeofloss'].isnull().sum().sum()

In [ ]:
#drop columns with NaN in causeofloss
fw_loss = fw[fw['causeofloss'].notna()]
fw_loss.shape

fw_loss

In [ ]:
#value counts for each country
fw_loss['country'].value_counts()

In [ ]:
#including only countries with 20 or more data points:
#- China, Benin, Ethiopia, Ghana, India, Iran, Kenya, Korea, Malawi, Bangladesh, 
#Nepal, New Zealand, Nigeria, Pakistan, Philippines, Rwanda, UK, Tanzania

###Subset dataset so that only the above 18 countries data are present. Save this to a new datafram object fw_final

places = ['China', 'Benin', 'Ethiopia', 'Ghana', 'India', 'Iran (Islamic Republic of)', 'Kenya', 'Republic of Korea', 'Malawi', 'Bangladesh', 'Nepal', 'New Zealand', 'Nigeria', 'Pakistan', 'Philippines', 'Rwanda', 'United Kingdom', 'United Republic of Tanzania']

fw_final = fw_loss[fw_loss.country.isin(places)]

fw_final.shape #935 x 22

In [ ]:
####Based on the number of null values or relevance to question, deciding which variables should be removed

fw_loss['region'].isnull().sum().sum() #766---------------------------REMOVE
fw_loss['measureditemcpc'].isnull().sum().sum() #0---------keep
fw_loss['loss_quantity'].isnull().sum().sum() #1040-------------------REMOVE
fw_loss['loss_qualitiative'].isnull().sum().sum() #1057---------------REMOVE
fw_loss['loss_monetary'].isnull().sum().sum() #1089-------------------REMOVE
fw_loss['activity'].isnull().sum().sum() #812-----don't know
fw_loss['fsc_location1'].isnull().sum().sum() #45----------keep
fw_loss['periodofstorage'].isnull().sum().sum() #1013-----------------REMOVE
fw_loss['treatment'].isnull().sum().sum() #1128-----------------------REMOVE
fw_loss['causeofloss'].isnull().sum().sum() #0-------------keep
fw_loss['samplesize'].isnull().sum().sum() #599
fw_loss['units'].isnull().sum().sum() #1128---------------------------REMOVE
fw_loss['method_datacollection'].isnull().sum().sum() #568
fw_loss['tag_datacollection'].isnull().sum().sum() #0-----------------REMOVE
fw_loss['reference'].isnull().sum().sum() #138------------------------REMOVE
fw_loss['url'].isnull().sum().sum() #164------------------------------REMOVE
fw_loss['crop'].isnull().sum().sum() #0 ------------------keep
fw_loss['timepointyears'].isnull().sum().sum() #0---------keep
fw_loss['loss_per_clean'].isnull().sum().sum() #0---------keep
fw_loss['percentage_loss_of_quantity'].isnull().sum().sum() #1----keep

In [ ]:
#removed the following variables
fw_final = fw_final.drop(['region', 'loss_quantity', 'loss_qualitiative', 'loss_monetary', 'periodofstorage', 'treatment', 'units', 'tag_datacollection', 'reference', 'url', 'geographicaream49'], axis = 1)

In [ ]:
fw_final.shape #after removing columns shape is 935 x 12. so the 10 columns have been removed

In [ ]:
#Looking at ther variables to remove based on number of Null values
fw_final['activity'].isnull().sum().sum() #647-------------------remove
fw_final['samplesize'].isnull().sum().sum() #492-----------------remove
fw_final['method_datacollection'].isnull().sum().sum() #452------remove

In [ ]:
#removed the above ones
fw_final = fw_final.drop(['activity', 'samplesize', 'method_datacollection'], axis = 1)

In [ ]:
#checking shape to confirm changes were made to fw_final
fw_final.shape

In [ ]:
#remove %loss quantity and rename loss_per_clean as they are the same data points but loss_per_clean is cleaner. 
fw_final = fw_final.drop(['percentage_loss_of_quantity'], axis = 1)
fw_final = fw_final.drop(['measureditemcpc'], axis = 1) #categorization for crops, we'll encode instead

fw_final.shape

In [ ]:
#rename loss_per_clean to percentage_loss_of_quant
fw_final.rename(columns={'loss_per_clean': 'percentage_loss_of_quant'}, inplace=True)
fw_final.head()#to check if change happened

<h5>Getting counts for each category in the features</h5>

In [ ]:
#for country
fw_final['country'].value_counts()